# Extracción de datos de la API del BCRP.
La Base de Datos de Estadísticas del BCRP (BCRPData) proporciona la herramienta API para Desarrolladores como una de las facilidades que se brindan para la obtención y uso de las series macroeconómicas. (https://estadisticas.bcrp.gob.pe/estadisticas/series/documentos/bcrpdataapi.pdf)

Este código costruira una URL que se utilizará para acceder a datos de estadísticas proporcionados por el Banco Central de Reserva del Perú (BCRP) a través de su API. 

In [1]:
url_base="https://estadisticas.bcrp.gob.pe/estadisticas/series/api/"
cod_ser="PN38063GM" # Tasa de Desempleo (%)
formato="/csv"
per="/2018-9/2023-9"
url=url_base+cod_ser+formato+per
print(url)
#args="address=1600+Amphitheatre+Parkway, +Mountain+View, +CA&"+key_api
#No necesitamos poner la url en formato de diccionario, no hay "?" no usamos query con argumentos con key y values

https://estadisticas.bcrp.gob.pe/estadisticas/series/api/PN38063GM/csv/2018-9/2023-9


Utilizando la biblioteca requests de Python para realizar una solicitud HTTP GET a la URL construida previamente

In [2]:
import requests 
response = requests.get(url)  # Response 200 todo OK 
print(response)

<Response [200]>


Después de la solicitud HTTP GET y procesamos la respuesta obtenida de la API del BCRP y colocamos los datos en un DF.

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# Comprueba si la solicitud se realizó con éxito (código de respuesta 200)
if response.status_code == 200:
    # Utiliza BeautifulSoup para eliminar el HTML y obtener el texto
    soup = BeautifulSoup(response.text, 'html.parser')
    data = soup.get_text()
    
    # Utiliza una expresión regular para extraer la fecha y el valor
    pattern = r'"(.*?)","(.*?)"'
    matches = re.findall(pattern, data)
    
    # Crea una lista de diccionarios para los datos
    data_list = []
    for match in matches:
        date = match[0].replace('"', '')  # Elimina las comillas
        value = float(match[1])
        data_list.append({'Fecha': date, 'Tasa de Desempleo (%)': value})
    
    # Crea un DataFrame a partir de la lista de diccionarios
    df = pd.DataFrame(data_list)
    
    # Imprime el DataFrame
    print(df)
else:
    print(f"La solicitud GET a la API falló con el código de respuesta: {response.status_code}")

       Fecha  Tasa de Desempleo (%)
0   Sep.2018               6.181888
1   Oct.2018               6.425933
2   Nov.2018               6.247634
3   Dic.2018               6.126948
4   Ene.2019               6.476252
..       ...                    ...
56  May.2023               6.778779
57  Jun.2023               6.551520
58  Jul.2023               6.264338
59  Ago.2023               6.637680
60  Sep.2023               6.731492

[61 rows x 2 columns]


Realizamos la conversión de fechas en un DataFrame, mapeando los nombres de los meses abreviados a números de mes en formato 'mm/dd/yyyy

In [4]:
# Se crea un diccionario llamado meses, que asocia los nombres abreviados de los meses con sus números correspondientes.
# Por ejemplo, 'Ene' se asigna a '01', 'Feb' a '02', y así sucesivamente.
meses = {
    'Ene': '01', 'Feb': '02', 'Mar': '03', 'Abr': '04',
    'May': '05', 'Jun': '06', 'Jul': '07', 'Ago': '08',
    'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dic': '12'
}

# Función para convertir las fechas
def convertir_fecha(fecha):
    
    """ Se define una función llamada convertir_fecha que toma una fecha en formato 'Mmm.dd' 
    (donde 'Mmm' es el nombre abreviado del mes y 'dd' es el día) y la convierte en '01/mm/dd'.
    Si el nombre del mes no se encuentra en el diccionario, se utiliza '01' como mes por defecto.
    """
    partes = fecha.split('.')
    mes = meses.get(partes[0], '01')  # Si el mes no se encuentra, se usa '01' como valor por defecto
    nueva_fecha = f'01/{mes}/{partes[1]}'
    return nueva_fecha

# Aplicar la función a la columna 'Fecha'
df['Fecha'] = df['Fecha'].apply(convertir_fecha)

# Mostrar el DataFrame resultante
print(df)

         Fecha  Tasa de Desempleo (%)
0   01/09/2018               6.181888
1   01/10/2018               6.425933
2   01/11/2018               6.247634
3   01/12/2018               6.126948
4   01/01/2019               6.476252
..         ...                    ...
56  01/05/2023               6.778779
57  01/06/2023               6.551520
58  01/07/2023               6.264338
59  01/08/2023               6.637680
60  01/09/2023               6.731492

[61 rows x 2 columns]


In [5]:
# Función para obtener un DataFrame a partir de una serie específica
def obtener_datos_economicos(cod_ser, columna_nombre):
    url_base = "https://estadisticas.bcrp.gob.pe/estadisticas/series/api/"
    formato = "/csv"
    periodo = "/2018-9/2023-9"
    url = url_base + cod_ser + formato + periodo

    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        data = soup.get_text()

        pattern = r'"(.*?)","(.*?)"'
        matches = re.findall(pattern, data)

        data_list = []
        for match in matches:
            date = match[0].replace('"', '')
            value = float(match[1])
            data_list.append({'Fecha': date, columna_nombre: value})

        df = pd.DataFrame(data_list)

        meses = {
            'Ene': '01', 'Feb': '02', 'Mar': '03', 'Abr': '04',
            'May': '05', 'Jun': '06', 'Jul': '07', 'Ago': '08',
            'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dic': '12'
        }

        def convertir_fecha(fecha):
            partes = fecha.split('.')
            mes = meses.get(partes[0], '01')
            nueva_fecha = f'01/{mes}/{partes[1]}'
            return nueva_fecha

        df['Fecha'] = df['Fecha'].apply(convertir_fecha)

        return df
    else:
        print(f"La solicitud GET a la API falló con el código de respuesta: {response.status_code}")
        return None


In [6]:
# PBI_1 = obtener_datos_economicos("PN01728AM", "PBI (variaciones porcentuales anualizadas)")
# PBI_2 = obtener_datos_economicos("PN01770AM", "PBI (índice 2007=100)")
# T_CAMBIO = obtener_datos_economicos("PN01234PM", "TIPO_CAMBIO (USD)")
# INFLACION_BIENES = obtener_datos_economicos("PN01296PM", "INFLACION_BIENES (variación porcentual)")
# INFLACION_SERVICIOS = obtener_datos_economicos("PN01301PM", "INFLACION_SERVICIOS (variación porcentual)")
# INFLACION_ALIMENTOS = obtener_datos_economicos("PN01308PM", "INFLACION_ALIMENTOS (variación porcentual)")
# INFLACION_COMBUSTIBLE = obtener_datos_economicos("PN01309PM", "INFLACION_COMBUSTIBLE (variación porcentual)")
# RESERVAS_INTERNAS = obtener_datos_economicos("PN00026MM", "RESERVAS_INTERNACIONALES_NETAS (millones S/)")
# EXPORTACIONES = obtener_datos_economicos("RD38111BM", "Exportaciones (Valores FOB en millones de US$)")
# EXPORTACIONES_TRADICIONALES = obtener_datos_economicos("PN38814BM", "Exportaciones de productos tradicionales - valores FOB (millones US$)")
# EXPORTACIONES_NO_TRADICIONALES = obtener_datos_economicos("PN38890BM", "Exportaciones de productos no tradicionales - valores FOB (millones US$)")
# CREDITO_PRIVADO = obtener_datos_economicos("PN00012MM", "Activos Internos Netos - Crédito al Sector Privado (millones S/)")
# LIQUIDES_BANCARIA = obtener_datos_economicos("PN00135MM", "Liquidez de las empresas bancarias (millones S/)")
# ACTIVAS_TAMN = obtener_datos_economicos("PN07807NM", "Tasas Activas para el Mercado Interbancario")
# DESEMPLEO_LIMA = obtener_datos_economicos("PN38063GM", "Desempleo en Lima Metropolitana - Promedio móvil tres meses (porcentaje)")


In [7]:
indicadores = [
    ("PBI_1", "PN01728AM", "PBI (variaciones porcentuales anualizadas)"),
    ("PBI_2", "PN01770AM", "PBI (índice 2007=100)"),
    ("T_CAMBIO", "PN01234PM", "TIPO_CAMBIO (USD)"),
    ("INFLACION_BIENES", "PN01296PM", "INFLACION_BIENES (variación porcentual)"),
    ("INFLACION_SERVICIOS", "PN01301PM", "INFLACION_SERVICIOS (variación porcentual)"),
    ("INFLACION_ALIMENTOS", "PN01308PM", "INFLACION_ALIMENTOS (variación porcentual)"),
    ("INFLACION_COMBUSTIBLE", "PN01309PM", "INFLACION_COMBUSTIBLE (variación porcentual)"),
    ("RESERVAS_INTERNAS", "PN00026MM", "RESERVAS_INTERNACIONALES_NETAS (millones S/)"),
    ("EXPORTACIONES", "RD38111BM", "Exportaciones (Valores FOB en millones de US$)"),
    ("EXPORTACIONES_TRADICIONALES", "PN38814BM", "Exportaciones de productos tradicionales - valores FOB (millones US$)"),
    ("EXPORTACIONES_NO_TRADICIONALES", "PN38890BM", "Exportaciones de productos no tradicionales - valores FOB (millones US$)"),
    ("CREDITO_PRIVADO", "PN00012MM", "Activos Internos Netos - Crédito al Sector Privado (millones S/)"),
    ("LIQUIDES_BANCARIA", "PN00135MM", "Liquidez de las empresas bancarias (millones S/)"),
    ("ACTIVAS_TAMN", "PN07807NM", "Tasas Activas para el Mercado Interbancario"),
    ("DESEMPLEO_LIMA", "PN38063GM", "Desempleo en Lima Metropolitana - Promedio móvil tres meses (porcentaje)")
]

for variable, codigo, descripcion in indicadores:
    globals()[variable] = obtener_datos_economicos(codigo, descripcion)


In [10]:
import pandas as pd

# Supongamos que PBI_1, PBI_2, T_CAMBIO, ... son DataFrames con la columna "Fecha"
# Asegúrate de que todos los DataFrames tengan una columna llamada "Fecha"

# Primero, inicializa un DataFrame con la columna "Fecha" que servirá como base
base_df = PBI_1[["Fecha"]]

# Luego, realiza la unión de los DataFrames utilizando la columna "Fecha"
dataframes = [PBI_1, PBI_2, T_CAMBIO, INFLACION_BIENES, INFLACION_SERVICIOS, 
              INFLACION_ALIMENTOS, INFLACION_COMBUSTIBLE, RESERVAS_INTERNAS, 
              EXPORTACIONES, EXPORTACIONES_TRADICIONALES, EXPORTACIONES_NO_TRADICIONALES, 
              CREDITO_PRIVADO, LIQUIDES_BANCARIA, ACTIVAS_TAMN, DESEMPLEO_LIMA]

for df in dataframes:
    base_df = base_df.merge(df, on="Fecha", how="outer")

# El resultado final estará en base_df


In [11]:
base_df

,Fecha,PBI (variaciones porcentuales anualizadas),PBI (índice 2007=100),TIPO_CAMBIO (USD),INFLACION_BIENES (variación porcentual),INFLACION_SERVICIOS (variación porcentual),INFLACION_ALIMENTOS (variación porcentual),INFLACION_COMBUSTIBLE (variación porcentual),RESERVAS_INTERNACIONALES_NETAS (millones S/),Exportaciones (Valores FOB en millones de US$),Exportaciones de productos tradicionales - valores FOB (millones US$),Exportaciones de productos no tradicionales - valores FOB (millones US$),Activos Internos Netos - Crédito al Sector Privado (millones S/),Liquidez de las empresas bancarias (millones S/),Tasas Activas para el Mercado Interbancario,Desempleo en Lima Metropolitana - Promedio móvil tres meses (porcentaje)
0,01/01/2018,2.398056,NaN,NaN,NaN,NaN,NaN,NaN,191203.685607,3885.874262,2822.906101,1047.6470,NaN,220981.58170,NaN,NaN
1,01/10/2018,4.090962,171.603442,3.333909,0.099473,0.088182,-0.011169,1.019750,197118.797205,3989.093544,2775.025687,1190.1961,304562.982118,222712.17476,14.1619,6.425933
2,01/11/2018,5.063620,170.240885,3.374675,0.110047,0.087123,0.441894,0.148204,200728.244582,4169.265158,3004.242039,1137.8181,307709.805871,227918.87176,14.2185,6.247634
3,01/12/2018,4.709017,187.229533,3.364026,0.139788,0.109914,-0.293831,-0.850696,202608.673882,4322.300665,3046.495574,1255.4050,311367.884525,229060.27822,14.2958,6.126948
4,01/01/2019,1.674375,157.186802,3.343864,0.080862,0.174589,-0.543605,-0.680250,207724.052264,3943.293905,2688.523614,1241.6653,309510.830610,232861.46554,14.3210,6.476252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,01/06/2023,-0.559386,177.077082,3.651048,0.207632,0.377723,-2.112935,-1.852119,264782.406756,NaN,4312.671059,1278.9505,412757.384849,305911.86361,15.5390,6.551520
58,01/07/2023,-1.290000,175.146332,3.601300,0.144270,0.447371,1.083284,-2.296622,264521.150243,NaN,3820.026297,1343.0488,412470.367363,309519.13974,15.6423,6.264338
59,01/08/2023,-0.630281,177.628686,3.696341,0.277554,0.333839,1.322426,3.794376,265136.483500,NaN,4002.685207,1446.2080,415825.546997,310887.42648,15.7516,6.637680
60,01/09/2018,NaN,168.105837,3.311325,0.123563,0.101517,0.907554,0.260885,NaN,NaN,NaN,NaN,301011.036876,NaN,14.3020,6.181888


In [13]:
import pandas as pd

# Supongamos que tienes un DataFrame llamado 'dataframe'
# Puedes verificar los valores nulos por columnas de la siguiente manera:

null_counts = base_df.isnull().sum()
print(null_counts)


Fecha                                                                        0
PBI (variaciones porcentuales anualizadas)                                   2
PBI (índice 2007=100)                                                        2
TIPO_CAMBIO (USD)                                                            1
INFLACION_BIENES (variación porcentual)                                      1
INFLACION_SERVICIOS (variación porcentual)                                   1
INFLACION_ALIMENTOS (variación porcentual)                                   1
INFLACION_COMBUSTIBLE (variación porcentual)                                 1
RESERVAS_INTERNACIONALES_NETAS (millones S/)                                 1
Exportaciones (Valores FOB en millones de US$)                              22
Exportaciones de productos tradicionales - valores FOB (millones US$)        2
Exportaciones de productos no tradicionales - valores FOB (millones US$)     2
Activos Internos Netos - Crédito al Sector Privado (